In [8]:
import sys

from pathlib import Path
main_path = Path().absolute().parent
data_path = main_path / 'data'
setting_path = main_path / 'setting_files'

sys.path.append(str(main_path))

import torch
import yaml
import json
import pytorch_lightning as pl
from pytorch_lightning import seed_everything

from src.nlu_models import NLUModel
from src.nlu_utils import NLUDataModule

with (setting_path / 'train_settings.yml').open('r') as file:
    settings = yaml.load(file, Loader=yaml.FullLoader)

data_module_settings = settings['data_module']
model_settings = settings['model']
trainer_settings = settings['trainer']

data_module = NLUDataModule(
    train_path=data_path / data_module_settings['train_file'], 
    valid_path=data_path / data_module_settings['valid_file'],
    test_path=data_path / data_module_settings['test_file'],
    labels_path=data_path / data_module_settings['labels_file'],
    batch_size=data_module_settings['batch_size'], 
    max_len=data_module_settings['max_len'],
    num_workers=data_module_settings['num_workers'],
    seed=settings['seed']
)

if model_settings.get('weight_file') is not None:
    with (data_path / model_settings['weight_file']).open('r', encoding='utf-8') as file:
        weight_dict = json.load(file)
else:
    weight_dict = None

hparams = {
    'stage': model_settings['stage'],
    'model_path': model_settings['model_path'], 
    'intent_size': len(data_module.intents2id), 
    'tags_size': len(data_module.tags2id), 
    'lr': model_settings['lr'],
    'weight_decay_rate': model_settings['weight_decay_rate'],
    'loss_type': model_settings['loss_type'],
    'multigpu': True if trainer_settings['n_gpus'] > 1 else False,
    'weight_dict': weight_dict
}
for k, v in model_settings['schedular'].items():
    hparams[f'schedular_{k}'] = v
if model_settings['loss_type'] == 'focal':
    for k, v in model_settings['focal'].items():
        hparams[f'focal_{k}'] = v

model_dict = {
    0: '_l7_tk3_ce.ckpt',
    1: '_l7_tk3_focal.ckpt',
    2: '_l8_tk3_focal.ckpt'
}

# checkpoint_path = str(main_path / 'checkpoints' / 'nlu_simple' / 'best_model.ckpt')

# seed_everything(seed=settings['seed'])
# trainer = pl.Trainer(
#     gpus=trainer_settings['n_gpus'], 
#     max_epochs=trainer_settings['n_epochs'], 
#     num_sanity_val_steps=trainer_settings['num_sanity_val_steps'],
#     deterministic=True,
# )
# # checkpoint = torch.load(checkpoint_path)
# model = NLUModel.load_from_checkpoint(checkpoint_path)
# # model.load_state_dict(checkpoint['state_dict'])
# model.eval()
# print()

In [1]:
'x_ce_l7_tk3_0.yml'.rstrip('.yml').split('_', 1)[-1]

'ce_l7_tk3_0'

In [3]:
from src.nlu_utils import NLUTokenizer

tokenizer = NLUTokenizer()

In [4]:
def predict(model, tokenizer, text):
    bert_encodes = tokenizer(
        text, 
        add_special_tokens=True, 
        truncation=True, 
        max_length=256,
        return_tensors='pt'
    )
    output = model.predict(**bert_encodes)
    return output

In [5]:
data = {"text": "current assets"}
text = data['text']
bert_encodes = tokenizer(
    text, 
    add_special_tokens=True, 
    truncation=True, 
    max_length=256,
    return_tensors='pt'
)
# o = predict(model, tokenizer, text)
# o

In [6]:
o = model(**bert_encodes)

In [11]:
o['intent'].argmax()

tensor(0)